In [1]:
import os
from os import path, listdir
import tensorflow as tf

import tensorflow as tf

import waymo_open_dataset
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

import utils
from utils import extract_and_serialize_frame, deserialize_example

import matplotlib.pyplot as plt

import numpy as np

import tarfile

In [14]:
def convert_waymo_data(tar_dir, temp_dir, tgt_dir, nb_tar):
    tars = [path.join(tar_dir, file) for file in listdir(tar_dir)]

    for (idx, tar) in enumerate(tars):
        if idx >= nb_tar:
            break
        print(f"Extracting {tar} to {temp_dir}")
        tar_file = tarfile.open(tar, 'r:')
        tar_file.extractall(temp_dir)
        tar_file.close()

        parsed_file = path.join(tgt_dir, 'parsed-' + str(idx).rjust(3, '0') + '.tfrecord')
        license_file = f'{temp_dir}/LICENSE'

        print(f'Deleting LICENSE file {temp_dir}/LICENSE')
        if path.exists(license_file):
            os.remove(license_file)
        temp_files = [f'{temp_dir}/{file}' for file in listdir(temp_dir)]

        print(f"Parsing data and writing to {parsed_file}")
        dataset = tf.data.TFRecordDataset(temp_files)
        writer = tf.io.TFRecordWriter(parsed_file)
        for (idx, data) in enumerate(dataset):
            if idx % 100 == 0:
                print(f'Processed {idx} frames')
            converted = extract_and_serialize_frame(data)
            writer.write(converted)
        writer.close()

        print(f"Deleting temp files from {temp_dir}")
        for file in temp_files:
            os.remove(file)

In [15]:
convert_waymo_data('Data/training/tar', 'Data/training/unprocessed', 'Data/training/parsed', 15)


Deleting LICENSE file Data/training/unprocessed/LICENSE
Parsing data and writing to Data/training/parsed/parsed-000.tfrecord
Deleting temp files from Data/training/unprocessed
